In [ ]:
import serial
import socket
import cv2
import ipywidgets as widgets
import traitlets
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
import jetson_inference
import jetson_utils
import threading
import time
from jetracer.nvidia_racecar import NvidiaRacecar

# Initialize the object detection network
net = jetson_inference.detectNet("ssd-mobilenet-v2", threshold=0.5)

# Initialize the camera
camera = CSICamera(width=224, height=224)
camera.running = True

# Create image preview widgets
camera_widget = widgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# Initialize the car
car = NvidiaRacecar()

# Increase throttle gain for more power
car.throttle_gain = 1.0  # Adjust as needed

# Car parameters
throttle = -0.8
speed_cm_per_second = 75  # Speed of the car in cm per second
segment_length = 30  # Length of each segment in cm

# Initial position
position = (0, 0)  # Starting point (6,0)

# Central server details
central_server_ip = "172.20.10.2"
central_server_port = 5005

# Function to move the car forward
def move_forward():
    print("Moving forward with increased throttle...")
    car.throttle = throttle

# Function to move the car backward
def move_backward():
    print("Moving backward with increased throttle...")
    car.throttle = -throttle  # Set throttle to negative for backward movement

# Function to stop the car
def stop():
    print("Stopping the car...")
    car.throttle = 0

# Function to calculate and update the car's position
def navigate(target_x):
    global position
    x, y = position
    start_time = time.time()
    
    print(f"Starting navigation from position {position} to ({target_x}, {y})...")
    
    if target_x > x:
        move_forward()
    else:
        move_backward()
    
    try:
        while True:
            current_time = time.time() - start_time
            distance_traveled = speed_cm_per_second * current_time
            segments_traveled = int(distance_traveled / segment_length)
            
            if target_x > x:  # Moving forward
                new_x = x + segments_traveled * segment_length
                if new_x >= target_x:
                    x = target_x
                    print(f"Updated position: ({x}, {y}), Time elapsed: {current_time:.2f} seconds")
                    break
                else:
                    x = new_x
            elif target_x < x:  # Moving backward
                new_x = x - segments_traveled * segment_length
                if new_x <= target_x:
                    x = target_x
                    print(f"Updated position: ({x}, {y}), Time elapsed: {current_time:.2f} seconds")
                    break
                else:
                    x = new_x
            
            # Print position and time elapsed
            print(f"Updated position: ({x}, {y}), Time elapsed: {current_time:.2f} seconds")
            
            # Sleep for a short period to control the update rate
            time.sleep(1)  # Update every second
    
    except KeyboardInterrupt:
        stop()
        print("Navigation stopped.")
    finally:
        stop()
        print(f"Final position: ({x}, {y})")
        
        # Update the global position to the new coordinate
        position = (x, y)
        
        # Check for a person at the target position
        check_for_person()

# Function to check for a person at the current position
def check_for_person():
    global position
    print("Checking for a person at the current position...")
    
    # Capture and process the image from the camera
    img = camera.value.copy()
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Convert to BGR format

    # Convert image to CUDA format for detection
    img_cuda = jetson_utils.cudaFromNumpy(img)
    
    # Perform object detection
    detections = net.Detect(img_cuda)
    img_cuda = jetson_utils.cudaToNumpy(img_cuda)
    img = cv2.cvtColor(img_cuda, cv2.COLOR_BGR2RGB)  # Convert back to RGB format

    person_detected = False
    for detection in detections:
        if net.GetClassDesc(detection.ClassID) == "person":
            person_detected = True
            print("A person is detected!")
            # Send coordinates and message to the central server
            send_message_to_central_server(position)
            break
    
    if not person_detected:
        print("No person detected at the current position.")

# Function to send message to central server
def send_message_to_central_server(position):
    message = f"Person detected at coordinate: {position}"
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.sendto(message.encode(), (central_server_ip, central_server_port))
    print(f"Message sent to central server: {message}")

# Function to handle localisation based on received coordinates
def localisation_function(coordinates):
    global position
    print(f"Localising to coordinates: {coordinates}")
    
    # Parse coordinates
    target_x, target_y = map(int, coordinates.split(','))
    
    if target_y == position[1]:  # Move only if y-coordinates match
        navigate(target_x)
    else:
        print("Target y-coordinate is not aligned with current y-coordinate. No movement.")

# Function to update the image with object detection
def update_image():
    last_frame = None  # Store the last frame with detections
    while True:
        # Capture and process the image from the camera
        img = camera.value.copy()
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Convert to BGR format

        # Convert image to CUDA format for detection
        img_cuda = jetson_utils.cudaFromNumpy(img)
        
        # Perform object detection
        detections = net.Detect(img_cuda)
        img_cuda = jetson_utils.cudaToNumpy(img_cuda)
        img = cv2.cvtColor(img_cuda, cv2.COLOR_BGR2RGB)  # Convert back to RGB format

        # Draw detection results
        person_detected = False
        for detection in detections:
            x1, y1, x2, y2 = int(detection.Left), int(detection.Top), int(detection.Right), int(detection.Bottom)
            label = net.GetClassDesc(detection.ClassID)
            color = (0, 255, 0)  # Green color for bounding boxes
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            if label == "person":
                person_detected = True

        if person_detected:
            print("A person is detected around here.")
            message = f"Person detected, latest coordinate: {position}"
            send_message_to_central_server(message)

        # Update the image widget
        if last_frame is not None:
            camera_widget.value = bgr8_to_jpeg(img)

        # Sleep for a short period to control the update rate
        time.sleep(0.5)  # 500 milliseconds

# Function to start object detection
def start_object_detection():
    print("Starting object detection...")
    display(camera_widget)
    update_thread = threading.Thread(target=update_image, daemon=True)
    update_thread.start()

# Function to listen for coordinates from other robots
def listen_for_coordinates():
    udp_ip_r = "0.0.0.0"  # Listen on all available interfaces
    udp_port_r = 5076     # Port number to listen on
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind((udp_ip_r, udp_port_r))

    while True:
        data, addr = sock.recvfrom(1024)  # Buffer size is 1024 bytes
        message = data.decode('utf-8')
        print(f"Received message from {addr}: {message}")
        
        # Check for specific messages and coordinates
        if message == "possible human detection":
            print("Possible human detection received, waiting for coordinates...")
            data, addr = sock.recvfrom(1024)  # Buffer size is 1024 bytes
            coordinates = data.decode('utf-8')
            print(f"Received coordinates: {coordinates}")
            localisation_function(coordinates)

# Function to start all operations concurrently
def start_operations_functions():
    # Start object detection
    start_object_detection()
    
    # Start listening for coordinates
    coordinate_thread = threading.Thread(target=listen_for_coordinates, daemon=True)
    coordinate_thread.start()

# Configure serial communication with Arduino
arduino_port = "/dev/ttyACM0"  # Adjust to your actual port
baud_rate = 9600
ser = serial.Serial(arduino_port, baud_rate, timeout=1)

# Configure UDP socket for communication with Robot 2
udp_ip = "172.20.10.5"  # IP address of Robot 2, 192.168.1.124 'HOME WIFI' 172.20.10.5 'Hotspor'
udp_port = 5057           # Port number on Robot 2
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

print("START! Waiting for data from Arduino...")

while True:
    line = ser.readline().decode('utf-8').strip()
    
    # Print all data received from Arduino
    if line:
        print(line)
    
    # Check for emergency message
    if line == "Emergency":
        print("Emergency detected. Sending message to Robot 2.")
        message = "EMERGENCY"
        sock.sendto(message.encode(), (udp_ip, udp_port))
        
        # Start all operations concurrently
        start_operations_functions()
        # No break here, the program will continue running the operations

START! Waiting for data from Arduino...
Found SGP30 sensor
TVOC: 0 ppb	CO2: 400 ppm
Flame detected!
Emergency
Emergency detected. Sending message to Robot 2.
Starting object detection...


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Received message from ('172.20.10.5', 38839): possible human detection
Possible human detection received, waiting for coordinates...
A person is detected around here.
Message sent to central server: Person detected at coordinate: Person detected, latest coordinate: (0, 0)
Received coordinates: 2,0
Localising to coordinates: 2,0
Starting navigation from position (0, 0) to (2, 0)...
Moving forward with increased throttle...
Updated position: (0, 0), Time elapsed: 0.01 seconds
Updated position: (2, 0), Time elapsed: 1.01 seconds
Stopping the car...
Final position: (2, 0)
Checking for a person at the current position...
A person is detected!
Message sent to central server: Person detected at coordinate: (2, 0)
A person is detected around here.
Message sent to central server: Person detected at coordinate: Person detected, latest coordinate: (2, 0)
A person is detected around here.
Message sent to central server: Person detected at coordinate: Person detected, latest coordinate: (2, 0)
A pe